# TL;DR – Too Long, Doctor

TL;DR is a ML model designed to synthesize and cluster scientific papers. Tailored for both students and researchers seeking to optimize their study time, TL;DR provides a tool to quickly grasp the essence of complex scientific material. Additionally, it caters to those who desire a concise summary or a preliminary overview of a paper before delving into a detailed reading.

## Importing libraries

In [1]:
# Import library to extract data from XML file
import xml.etree.ElementTree as ET
import os
import json
import re
import logging
import string

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BartTokenizer, BartForConditionalGeneration, AdamW, pipeline
from tqdm.auto import tqdm

In [3]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Dataset Generation (from XML to JSON)

In [4]:
def extract_information_from_xml(xml_path):
    """
    Parameters:
    xml_path (str): Path to the XML file.

    Returns:
    dict: A dictionary containing the extracted information, with separated abstract sub-layers.
    """
    try:
        # Parse the XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Initialize a dictionary to hold the extracted information
        extracted_info = {
            'Title': '',
            'Abstract': {'Simple Summary': '', 'Detailed Abstract': ''},
            'Sections': [],
            'Keywords': []
        }

        # Extract Title
        title_element = root.find('.//article-title')
        if title_element is not None:
            extracted_info['Title'] = ''.join(title_element.itertext())

        # Extract Abstracts
        abstract_element = root.find('.//abstract')
        if abstract_element is not None:
            sec_elements = abstract_element.findall('.//sec')
            for sec in sec_elements:
                section_title = ''.join(sec.find('.//title').itertext()).strip() if sec.find('.//title') is not None else ""
                section_text = ''.join(sec.itertext()).strip()
                
                # Remove the section title from the beginning of the section text
                if section_text.startswith(section_title):
                    section_text = section_text[len(section_title):].strip()
                
                if 'simple summary' in section_title.lower():
                    extracted_info['Abstract']['Simple Summary'] = section_text
                else:
                    # Append other sections to the 'Detailed Abstract', removing repeated titles if present
                    if extracted_info['Abstract']['Detailed Abstract']:
                        extracted_info['Abstract']['Detailed Abstract'] += ' ' + section_text
                    else:
                        extracted_info['Abstract']['Detailed Abstract'] = section_text
            
        # Extract Keywords
        kwd_group_elements = root.findall('.//kwd-group')
        for kwd_group in kwd_group_elements:
            keywords = [kwd.text for kwd in kwd_group.findall('.//kwd')]
            extracted_info['Keywords'].extend(keywords)

        # Extract Sections
        sections = root.findall('.//body//sec')
        for sec in sections:
            section_title_element = sec.find('.//title')
            if section_title_element is not None:
                section_title = ''.join(section_title_element.itertext())
                # Remove the title element to avoid repetition in the content
                sec.remove(section_title_element)
            else:
                section_title = "No Title"
            
            # Extracting content after removing the title
            section_content = ''.join(sec.itertext()).strip()
            extracted_info['Sections'].append({
                'Title': section_title,
                'Content': section_content
            })

        return extracted_info
    except ET.ParseError as e:
        print(f"XML parsing error: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

In [23]:
def extract_information_and_write_to_json_file(xml_path, output_directory):
    """
    Extracts Title, Abstract, Body, and Keyword Group from scientific papers in XML format and writes the information to a JSON file.
    The JSON file will have the same name as the XML file, but with a .json extension.
    
    Parameters:
    xml_path (str): Path to the XML file.
    """
    try:
        # Use the previously defined function to extract information
        extracted_info = extract_information_from_xml(xml_path)

        target_directory = output_directory
        # target_directory = './data/json/'
        os.makedirs(target_directory, exist_ok=True)

        # Generate the output JSON file path by changing the extension
        base_filename = os.path.basename(os.path.splitext(xml_path)[0])
        output_json_path = os.path.join(target_directory, base_filename + '.json')

        # Write the dictionary to a JSON file
        with open(output_json_path, 'w', encoding='utf-8') as json_file:
            json.dump(extracted_info, json_file, ensure_ascii=False, indent=4)
        
        print(f"JSON file has been written to {output_json_path}")
    except Exception as e:
        print(f"Error during JSON file writing: {e}")

xml_directory = "./data/xml/"
output_directory = "./data/json/"

# List all the XML files in the directory
xml_files = [f for f in os.listdir(xml_directory) if f.endswith('.xml')]

# Loop through each file and process it
for xml_file in xml_files:
    extract_information_and_write_to_json_file(xml_directory+xml_file, output_directory)

JSON file has been written to ./data/json/PMC10000009.json
JSON file has been written to ./data/json/PMC10000035.json
JSON file has been written to ./data/json/PMC10000034.json
JSON file has been written to ./data/json/PMC10000036.json
JSON file has been written to ./data/json/PMC10000022.json
JSON file has been written to ./data/json/PMC10000023.json
JSON file has been written to ./data/json/PMC10000037.json
JSON file has been written to ./data/json/PMC10000033.json
JSON file has been written to ./data/json/PMC10000027.json
JSON file has been written to ./data/json/PMC10000026.json
JSON file has been written to ./data/json/PMC10000032.json
JSON file has been written to ./data/json/PMC10000024.json
JSON file has been written to ./data/json/PMC10000030.json
JSON file has been written to ./data/json/PMC10000031.json
JSON file has been written to ./data/json/PMC10000025.json
JSON file has been written to ./data/json/PMC10000081.json
JSON file has been written to ./data/json/PMC10000095.js

## Preprocessing (from JSON to cleaned JSON that can be given to train the model)

In [6]:
def clean_html_tags(text):
    """
    Remove HTML/XML tags from the given text.
    
    Parameters:
    - text (str): Text to clean.
    
    Returns:
    str: Text with HTML/XML tags removed.
    """
    clean_text = re.sub(r'<[^>]+>', '', text)  # Remove anything within angle brackets
    return clean_text

In [7]:
def standardize_special_characters(text):
    """
    Standardize special characters in the text, such as converting different types of quotation marks
    to a standard form, and optionally removing characters that are not beneficial for the model.
    
    Parameters:
    - text (str): Text to process.
    
    Returns:
    str: Text with standardized special characters.
    """
    # Standardize quotation marks and apostrophes
    text = text.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")
    
    # Remove or replace other special characters as needed, e.g.:
    text = text.replace('—', '-')  # Replace long dashes with short ones
    
    return text

In [8]:
def replace_figures_tables_references(text):
    """
    Replace or remove references to figures and tables in the text.
    
    Parameters:
    - text (str): Text to process.
    
    Returns:
    str: Text with references to figures and tables handled.
    """
    
    # This regex targets common patterns like Figure 1, Fig. 1, Table 1, etc.
    text_without_references = re.sub(r'(Figure|Fig\.|Table)\s+\d+', '', text)
    
    return text_without_references

In [9]:
def remove_bibliography_references(text):
    """
    Remove references to the bibliography in the text, typically formatted as [1] or [1,2] or [1-3].
    
    Parameters:
    - text (str): Text to process.
    
    Returns:
    str: Text with bibliography references removed.
    """
    # This regex matches patterns like [1], [1,2], [1-3], etc.
    text_without_references = re.sub(r'\[\d+(-\d+)?(,\d+(-\d+)?)*\]', '', text)
    
    return text_without_references

In [10]:
def normalize_whitespace(text):
    """
    Normalize the whitespace in the text by replacing multiple spaces or line breaks with a single space
    and trimming leading and trailing spaces.
    
    Parameters:
    - text (str): Text to process.
    
    Returns:
    str: Text with normalized whitespace.
    """
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Trim leading and trailing spaces
    text = text.strip()
    
    return text

In [11]:
def preprocess_json(original_json_path, output_folder):
    """
    Preprocess the given JSON file by combining all sections into a single body string.
    The result is saved in a specified output folder with the same filename.
    
    Parameters:
    - original_json_path: Path to the original JSON file.
    - output_folder: Folder where the processed JSON should be saved.
    """
    # Create the output folder if it does not exist

    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            
        with open(original_json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
    except Exception as e:
        logging.error(f"Error reading {original_json_path}: {e}")
        return None
        
    try:
        # Combine all section contents into a single string, separated by double newlines
        combined_sections = "\n\n".join([section['Content'] for section in data['Sections']])

        combined_sections = normalize_whitespace(remove_bibliography_references(replace_figures_tables_references(standardize_special_characters(clean_html_tags(combined_sections)))))

        # Preprocess Abstract
        cleaned_abstract = data['Abstract']['Detailed Abstract']
        cleaned_abstract = normalize_whitespace(remove_bibliography_references(replace_figures_tables_references(standardize_special_characters(clean_html_tags(cleaned_abstract)))))
        
        # Update the 'Sections' key to a single string containing all combined sections
        processed_data = {
            'Title': data['Title'],
            'Abstract': cleaned_abstract,
            'Body': combined_sections
        }
        
        # Add keywords if present
        if 'Keywords' in data:
            processed_data['Keywords'] = data['Keywords']

        # Construct the output path
        output_path = os.path.join(output_folder, os.path.basename(original_json_path))
        
        # Save the processed data to the new JSON file
        with open(output_path, 'w', encoding='utf-8') as outfile:
            json.dump(processed_data, outfile, ensure_ascii=False, indent=4)

        logging.info(f"Processed file saved to {output_path}")
    except Exception as e:
        logging.error(f"Error processing {original_json_path}: {e}")
        return None
    
    return output_path

In [12]:
json_directory = "./data/json/"
json_processed_output_directory = "./data/json_processed"

# List all the JSON files in the directory
json_files = [f for f in os.listdir(json_directory) if f.endswith('.json')]

# Loop through each file and process it
for json_file in json_files:
    preprocess_json(json_directory+json_file, json_processed_output_directory)

2024-03-08 11:48:34,053 - INFO - Processed file saved to ./data/json_processed/PMC10000054.json
2024-03-08 11:48:34,055 - INFO - Processed file saved to ./data/json_processed/PMC10000111.json
2024-03-08 11:48:34,056 - INFO - Processed file saved to ./data/json_processed/PMC10000003.json
2024-03-08 11:48:34,059 - INFO - Processed file saved to ./data/json_processed/PMC10000015.json
2024-03-08 11:48:34,065 - INFO - Processed file saved to ./data/json_processed/PMC10000042.json
2024-03-08 11:48:34,068 - INFO - Processed file saved to ./data/json_processed/PMC10000107.json
2024-03-08 11:48:34,071 - INFO - Processed file saved to ./data/json_processed/PMC10000039.json
2024-03-08 11:48:34,073 - INFO - Processed file saved to ./data/json_processed/PMC10000081.json
2024-03-08 11:48:34,077 - INFO - Processed file saved to ./data/json_processed/PMC10000097.json
2024-03-08 11:48:34,081 - INFO - Processed file saved to ./data/json_processed/PMC10000078.json
2024-03-08 11:48:34,084 - INFO - Process

## Preprocessing (from cleaned JSON to Model Input)
For each paper in the dataset, we want to tokenize both the body (as input) and the abstract (as the target summary) using the BART tokenizer. This involves converting the text to model-compatible input IDs. We'll also need to manage the token length due to model limitations.

In [13]:
def tokenize_for_bart(text, tokenizer, max_length=1024):
    """
    Tokenizes the text, ensuring it does not exceed the maximum length for BART.
    """
    # Tokenize and truncate or pad the sequence to be of 'max_length'
    tokens = tokenizer.encode_plus(text, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
    return tokens

The following custom dataset class takes a list of file paths (data_files), loads the data, and stores body-abstract pairs. When accessed, it returns tokenized versions of these pairs, suitable for training.

In [14]:
class SummarizationDataset(Dataset):
    def __init__(self, tokenizer, data_files, max_length=1024, max_target_length=128):
        self.tokenizer = tokenizer
        self.data = []
        self.max_length = max_length
        self.max_target_length = max_target_length

        for file_path in data_files:
            with open(file_path, 'r', encoding='utf-8') as f:
                paper = json.load(f)
                self.data.append((paper['Body'], paper['Abstract']))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        body_text, abstract_text = self.data[idx]

        # Tokenize body text
        body_tokens = self.tokenizer.encode_plus(
            body_text, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt"
        )

        # Tokenize abstract (summary)
        abstract_tokens = self.tokenizer.encode_plus(
            abstract_text, max_length=self.max_target_length, truncation=True, padding="max_length", return_tensors="pt"
        )

        return {
            "input_ids": body_tokens['input_ids'].squeeze(),  # Squeeze to remove batch dimension
            "attention_mask": body_tokens['attention_mask'].squeeze(),
            "labels": abstract_tokens['input_ids'].squeeze(),
            # BART uses -100 to ignore padding in labels
            "decoder_attention_mask": abstract_tokens['attention_mask'].squeeze()
        }

Assuming we have a list of JSON file paths we want to use for training (see previous cells), we create an instance of our dataset and then a DataLoader.

In [18]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

clean_json_directory = "./data/json_processed/"

# This code will list all JSON files in clean_json_directory and create full paths to each file. 
# These paths are then passed to the SummarizationDataset to be processed.
data_files = [os.path.join(clean_json_directory, f) for f in os.listdir(clean_json_directory) if f.endswith('.json')]

# The SummarizationDataset class is expected to loop over data_files, which should be a list of 
# file paths, and process each file by tokenizing its contents and preparing them for training the BART model. 
dataset = SummarizationDataset(tokenizer, data_files)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Prepare optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [19]:
# Training loop
model.train()
num_epochs = 3

for epoch in range(num_epochs):
    epoch_loss = 0
    for batch in tqdm(dataloader, desc=f"Training Epoch {epoch + 1}"):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(model.device)
        attention_mask = batch["attention_mask"].to(model.device)
        labels = batch["labels"].to(model.device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    print(f"Epoch {epoch + 1} completed. Average loss: {epoch_loss / len(dataloader)}")

Training Epoch 1:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1 completed. Average loss: 3.2945681548118593


Training Epoch 2:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 2 completed. Average loss: 1.7448062252998353


Training Epoch 3:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 3 completed. Average loss: 1.254610733985901


In [20]:
# Save the model
model.save_pretrained('./model/')
tokenizer.save_pretrained('./model/')

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.json',
 './model/merges.txt',
 './model/added_tokens.json')

## Evaluation

### Preparing the Test Set

In [26]:
xml_test_directory = "./data/test/xml/"
output_test_directory = "./data/test/json"

# List all the XML files in the directory
xml_test_files = [f for f in os.listdir(xml_test_directory) if f.endswith('.xml')]

# Loop through each file and process it
for xml_test_file in xml_test_files:
    extract_information_and_write_to_json_file(xml_test_directory+xml_test_file, output_test_directory)

JSON file has been written to ./data/test/json/PMC10000196.json
JSON file has been written to ./data/test/json/PMC10000182.json
JSON file has been written to ./data/test/json/PMC10000169.json
JSON file has been written to ./data/test/json/PMC10000155.json
JSON file has been written to ./data/test/json/PMC10000141.json
JSON file has been written to ./data/test/json/PMC10000140.json
JSON file has been written to ./data/test/json/PMC10000154.json
JSON file has been written to ./data/test/json/PMC10000168.json
JSON file has been written to ./data/test/json/PMC10000183.json
JSON file has been written to ./data/test/json/PMC10000197.json
JSON file has been written to ./data/test/json/PMC10000208.json
JSON file has been written to ./data/test/json/PMC10000181.json
JSON file has been written to ./data/test/json/PMC10000195.json
JSON file has been written to ./data/test/json/PMC10000142.json
JSON file has been written to ./data/test/json/PMC10000156.json
JSON file has been written to ./data/tes

In [27]:
json_test_directory = "./data/test/json/"
json_test_processed_output_directory = "./data/test/json_processed"

# List all the JSON files in the directory
json_test_files = [f for f in os.listdir(json_test_directory) if f.endswith('.json')]

# Loop through each file and process it
for json_test_file in json_test_files:
    preprocess_json(json_test_directory+json_test_file, json_test_processed_output_directory)

2024-03-08 15:14:12,207 - INFO - Processed file saved to ./data/test/json_processed/PMC10000146.json
2024-03-08 15:14:12,210 - INFO - Processed file saved to ./data/test/json_processed/PMC10000150.json
2024-03-08 15:14:12,212 - INFO - Processed file saved to ./data/test/json_processed/PMC10000193.json
2024-03-08 15:14:12,214 - INFO - Processed file saved to ./data/test/json_processed/PMC10000202.json
2024-03-08 15:14:12,218 - INFO - Processed file saved to ./data/test/json_processed/PMC10000185.json
2024-03-08 15:14:12,221 - INFO - Processed file saved to ./data/test/json_processed/PMC10000170.json
2024-03-08 15:14:12,228 - INFO - Processed file saved to ./data/test/json_processed/PMC10000127.json
2024-03-08 15:14:12,232 - INFO - Processed file saved to ./data/test/json_processed/PMC10000131.json
2024-03-08 15:14:12,236 - INFO - Processed file saved to ./data/test/json_processed/PMC10000166.json
2024-03-08 15:14:12,240 - INFO - Processed file saved to ./data/test/json_processed/PMC1000

We will use the 'SummarizationDataset' class we've created to process the test set. We'll change a little bit, because this time no labels are required, we don't have shuffling and we need to put the model in evaluation mode.

In [28]:
# Function to process the test set using the SummarizationDataset class
def process_test_set(tokenizer, test_files, batch_size=2):
    test_dataset = SummarizationDataset(tokenizer, test_files, max_length=1024, max_target_length=128)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return test_dataloader

# Get the list of file paths for your test JSON files
test_json_directory = "./data/test/json_processed/"
test_data_files = [os.path.join(test_json_directory, f) for f in os.listdir(test_json_directory) if f.endswith('.json')]

# Process the test set
test_dataloader = process_test_set(tokenizer, test_data_files)

# Model in evaluation mode
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

### Generate Summaries

In [29]:
# Loading the model
model = BartForConditionalGeneration.from_pretrained('./model/')

In [30]:
# Function to generate summaries for the test set with progress logging
def generate_summaries_with_logging(model, dataloader, device):
    model.to(device)
    model.eval()
    summaries = []
    actuals = []
    progress_bar = tqdm(dataloader, desc='Generating Summaries')
    
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            # Labels are actually the target summaries (abstracts)
            labels = batch['labels'].to(device)  

            # Generate summaries
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
            
            # Convert generated ids to text
            batch_summaries = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]
            summaries.extend(batch_summaries)
            
            # Convert labels ids to text for actual summaries
            batch_actuals = [tokenizer.decode(a, skip_special_tokens=True, clean_up_tokenization_spaces=False) for a in labels]
            actuals.extend(batch_actuals)

            # Update progress bar description with latest information
            progress_bar.set_description(f"Processed {len(summaries)} summaries")

    return summaries, actuals

In [31]:
# Use a CUDA device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate summaries with logging
generated_summaries, actual_abstracts = generate_summaries_with_logging(model, test_dataloader, device)

### Calculate ROUGE Scores

In [36]:
from rouge_score import rouge_scorer, scoring

# Function to calculate ROUGE scores
def calculate_rouge_scores(references, predictions):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()
    
    for reference, prediction in zip(references, predictions):
        scores = scorer.score(reference, prediction)
        aggregator.add_scores(scores)
    
    result = aggregator.aggregate()
    
    # Accessing the results
    for key in result.keys():
        print(f"{key}: F1: {result[key].mid.fmeasure:.3f}, Precision: {result[key].mid.precision:.3f}, Recall: {result[key].mid.recall:.3f}")

In [37]:
calculate_rouge_scores(actual_abstracts, generated_summaries)

2024-03-08 16:02:16,743 - INFO - Using default tokenizer.


rouge1: F1: 0.374, Precision: 0.363, Recall: 0.394
rouge2: F1: 0.120, Precision: 0.118, Recall: 0.125
rougeL: F1: 0.212, Precision: 0.207, Recall: 0.222


### Qualitative Testing

In [ ]:
import numpy as np

def process_json_file(file_path, tokenizer):
    with open(file_path, 'r', encoding='utf-8') as file:
        paper = json.load(file)
    body_text = paper['Body']  # Get the body text
    # Tokenize the body text
    inputs = tokenizer(body_text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
    return inputs

# Directory where the test JSON files are stored
test_json_directory = "./data/test/json_processed/"
# List all the JSON files in the directory
test_data_files = [os.path.join(test_json_directory, f) for f in os.listdir(test_json_directory) if f.endswith('.json')]

# Now process and tokenize a random file from the test set
random_paper_index = np.random.randint(len(test_data_files))
random_paper_path = test_data_files[random_paper_index]
tokenized_inputs = process_json_file(random_paper_path, tokenizer)

In [ ]:
# Load the model
model = BartForConditionalGeneration.from_pretrained('./model/')

def generate_summary(model, tokenizer, tokenized_inputs):
    model.eval()  # Put model in evaluation mode
    # Move to the appropriate device (CPU or GPU)
    tokenized_inputs = {key: value.to(model.device) for key, value in tokenized_inputs.items()}
    with torch.no_grad():
        summary_ids = model.generate(**tokenized_inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Generate the summary
generated_summary = generate_summary(model, tokenizer, tokenized_inputs)

# Load the actual abstract for comparison
with open(random_paper_path, 'r', encoding='utf-8') as file:
    actual_paper = json.load(file)
actual_abstract = actual_paper['Abstract']

# Output the generated summary and the actual abstract
print("Generated Summary:\n", generated_summary)
print("\nActual Abstract:\n", actual_abstract)